# Pipeline de Pré-processamento - Tic Tac Toe
**Author**: Gabriela Dellamora Paim

**Version**: 12/04/2025

**Python Ver**: 3.13.1

In [ ]:
import pandas as pd

PATH_OLD = './data_old/tic-tac-toe.data'
PATH_NEW = './data_processed.csv'
PATH_ON_GOING = './data_ongoing.csv'
X_WIN   = X = '1'
O_WIN   = O ='-1'
DRAW    = '0'
ONGOING = '0.5'

In [83]:
# Dataframe onde X sempre é quem começa
df = pd.read_csv(PATH_OLD, index_col=False)
df.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', 'category']
df.describe()

,0,1,2,3,4,5,6,7,8,category
count,957,957,957,957,957,957,957,957,957,957
unique,3,3,3,3,3,3,3,3,3,2
top,x,x,x,x,x,x,x,x,x,positive
freq,417,377,417,377,458,378,417,378,418,625


# Tratar Dados Existentes

In [84]:
# Para facilitar a diferenciacao entre X venceu e O venceu
df.replace(to_replace={'positive' : X_WIN}, inplace=True)

In [85]:
'''
Removemos as observações em que O inicia o jogo pois:
1 - Não segue as regras originais do jogo
2 - Vamos diminuir o escopo de observações devido a limitação de 32 observações DRAW
'''
# Observações onde O começa
# df_o_begins = df_x_begins.replace(to_replace={'o':'x', 'x':'o', X_WIN:O_WIN}).copy()

# df_finished_tables = pd.concat([df_x_begins, df_o_begins], ignore_index=True).copy()
# print(f'BEFORE={df_x_begins.shape} : AFTER={df_finished_tables.shape}')
# df_o_begins.describe()
print(df.category.unique())
df.describe()

['1' 'negative']


,0,1,2,3,4,5,6,7,8,category
count,957,957,957,957,957,957,957,957,957,957
unique,3,3,3,3,3,3,3,3,3,2
top,x,x,x,x,x,x,x,x,x,1
freq,417,377,417,377,458,378,417,378,418,625


In [86]:
def get_state(symbol):
    ''' Retorna simbolo utilizado pela pipeline.'''
    match symbol.lower():
        case 'o': return O_WIN
        case 'x': return X_WIN
    return DRAW

def check_wins(obs):
    '''Valida estado final do jogo. Pode ser O_WIN, X_WIN e DRAW. Trata casos que não precisam de alteração'''
    # lazy evaluation
    if obs.category != 'negative':
        return obs.category

    # Diagonais
    if (obs.iloc[0] == obs.iloc[4] == obs.iloc[8]) or (obs.iloc[2] == obs.iloc[4] == obs.iloc[6]):
        return get_state(obs.iloc[4])

    for i in range(3):
        # Linhas
        if obs.iloc[0 + i] == obs.iloc[3 + i] == obs.iloc[6 + i]:
            s = get_state(obs.iloc[0 + i])
            if s != DRAW:
                return s
        # Colunas
        if obs.iloc[0 + (i * 3)] == obs.iloc[1 + (i * 3)] == obs.iloc[2 + (i * 3)]:
            s = get_state(obs.iloc[0 + (i * 3)])
            if s != DRAW:
                return s
    return DRAW

In [87]:
df['category'] = df.apply(check_wins, axis=1)

In [88]:
print(df.category.unique())
print(f'Total={df.shape}')
print(f'Draw={df[df.category == DRAW].shape}')
df.describe()
df[df.category == DRAW].sample(10)

['1' '-1' '0']
Total=(957, 10)
Draw=(16, 10)


,0,1,2,3,4,5,6,7,8,category
954,o,x,o,x,o,x,x,o,x,0
948,x,o,x,o,o,x,x,x,o,0
952,o,x,x,x,o,o,o,x,x,0
956,o,o,x,x,x,o,o,x,x,0
946,x,o,x,x,o,o,o,x,x,0
945,x,o,x,x,o,x,o,x,o,0
942,x,x,o,o,o,x,x,x,o,0
953,o,x,o,x,x,o,x,o,x,0
950,o,x,x,x,x,o,o,o,x,0
947,x,o,x,o,x,x,o,x,o,0


In [89]:
# Exportar
df.to_csv(PATH_NEW, index=False)

In [90]:
df_ongoing = pd.read_csv(PATH_ON_GOING)
df_ongoing['category'] = '~'
df_ongoing = df_ongoing[['0', '1', '2', '3', '4', '5', '6', '7', '8', 'category']]

In [ ]:
df_concat = pd.concat([df, df_ongoing], ignore_index=True)
df_concat.to_csv('data_processed.csv', index=False)